In [ ]:
import numpy as np
import pickle
import cv2
from os import listdir
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
# from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing import image
from tensorflow.keras.utils import img_to_array
# from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
#Epochs
EPOCHS = 25
INIT_LR = 1e-3
BS = 32
# Height and Width of the images
default_img_size = tuple((256, 256))
img_size = 0
directory_root = r'C:\Users\Senka\Downloads\plantdisease_dataset'
width=256
height=256
depth=3

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Coverting images to an
def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None :
            image = cv2.resize(image, default_image_size)
            return img_to_array(image)
        else :
            return np.array([])
    except Exception as e:
        print(f"Error : {e}")
        return None

In [ ]:
image_list, label_list = [], []
try:
    print("Loading images ...")
    root_dir = listdir(directory_root)
    for directory in root_dir :
        # removing .DS_Store from list
        if directory == ".DS_Store" :
            root_dir.remove(directory)

    for plant_folder in root_dir :
        disease_folder_content = listdir(f"{directory_root}/{plant_folder}")

        for disease_folder in disease_folder_content :
            # removing .DS_Store from list
            if disease_folder == ".DS_Store" :
                disease_folder_content.remove(disease_folder)

        for plant_disease_folder in disease_folder_content:
            print(f"Processing {plant_disease_folder} ...")
            disease_img_list = listdir(f"{directory_root}/{plant_folder}/{plant_disease_folder}/")

            for single_plant_disease_image in disease_img_list :
                if single_plant_disease_image == ".DS_Store" :
                    disease_img_list.remove(single_plant_disease_image)
            # listing the entities of the folder
            for image in disease_img_list[:200]:
                image_directory = f"{directory_root}/{plant_folder}/{plant_disease_folder}/{image}"
                if image_directory.endswith(".jpg") == True or image_directory.endswith(".JPG") == True:
                    image_list.append(convert_image_to_array(image_directory))
                    label_list.append(plant_disease_folder)
    print("Image loading completed")
except Exception as e:
    print(f"Error : {e}")

Loading images ...
Error : [Errno 2] No such file or directory: 'C:\\Users\\Senka\\Downloads\\plantdisease_dataset'


In [ ]:
img_size = len(image_list) # length of arr

In [ ]:
label_binarizer = LabelBinarizer()
image_labels = label_binarizer.fit_transform(label_list)
# saving the file in .pkl format
pickle.dump(label_binarizer,open('label_transform.pkl', 'wb'))
n_classes = len(label_binarizer.classes_)

In [ ]:
print(label_binarizer.classes_) # This will print out the labels

In [ ]:
np_img_list = np.array(image_list, dtype=np.float16) / 225.0 # Coverting to an array

In [ ]:
# Splitting data as training and testing
print("Spliting data as train data and test data")
x_train, x_test, y_train, y_test = train_test_split(np_img_list, image_labels, test_size=0.2, random_state = 42) # 20% data is used for testing and remaining 80% for training

In [ ]:
# Augmentation of images while training the data
aug = ImageDataGenerator(
    rotation_range=25, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2,
    zoom_range=0.2,horizontal_flip=True,
    fill_mode="nearest")

In [ ]:
# Stacks up sequential layer in order of input to output
model = Sequential()
inputShape = (height, width, depth)
chanDim = -1
if K.image_data_format() == "channels_first":
    inputShape = (depth, height, width)
    chanDim = 1
model.add(Conv2D(32, (3, 3), padding="same",input_shape=inputShape))
model.add(Activation("relu")) # Mathematical function to determine the ouput of this network
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.25)) # Preventing from over fitting
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten()) # Flatten the data to one dimensional
model.add(Dense(1024))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(n_classes))
model.add(Activation("softmax"))

In [ ]:
# Summary
model.summary()

In [ ]:
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
# distribution
model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])
# train the network
print("training network...")

In [ ]:
history = model.fit_generator(
    aug.flow(x_train, y_train, batch_size=BS),
    validation_data=(x_test, y_test),
    steps_per_epoch=len(x_train) // BS,
    epochs=EPOCHS, verbose=1
    )

In [ ]:
accuracy = history.history['accuracy']
validation_accuracy = history.history['validation_accuracy']
loss = history.history['loss']
validation_loss = history.history['validation_loss']
epochs = range(1, len(accuracy) + 1)
#Training and validation accuracyuracy of the model
plt.plot(epochs, accuracy, 'b', label='Training accuracy')
plt.plot(epochs, validation_accuracy, 'r', label='Validation accuracy')
plt.title('Training and Validation accuracy')
plt.legend()

plt.figure()
#Training and validation loss of the model
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, validation_loss, 'r', label='Validation loss')
plt.title('Training and Validation loss')
plt.legend()
plt.show()

In [ ]:
print("Calculate the model accuracy")
scores = model.evaluate(x_test, y_test)
print(f"Test accuracy: {scores[1]*100}")

In [ ]:
print("Saving the model...")
pickle.dump(model,open('cnn_model.pkl', 'wb'))

In [ ]:
loaded_model = pickle.load(open('cnn_model.pkl', 'rb'))

In [ ]:
loaded_model = pickle.load(open('path\\cnn_model.pkl', 'rb'))

In [ ]:
# Testing the model
image_dir="path\\plantdisease_dataset\\PlantVillage\\Potato___Early_blight"

im=convert_image_to_array(image_dir)
np_image_li = np.array(im, dtype=np.float16) / 225.0
npp_image = np.expand_dims(np_image_li, axis=0)

In [ ]:
result=model.predict(npp_image)

print(result)

In [ ]:
index = np.where(result==np.max(result))
print("probability:"+str(np.max(result))+"\n"+label_binarizer.classes_[validation_loss[1][0]])